<a href="https://colab.research.google.com/github/CDU-data-science-team/zero-shot/blob/feature-Huggingface_transformer/Patient_Feedback_with_Hugging_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.5 MB 45.3 MB/s 
     |████████████████████████████████| 3.3 MB 51.6 MB/s 
     |████████████████████████████████| 895 kB 39.6 MB/s 


In [ ]:
from transformers import pipeline
from sklearn.metrics import accuracy_score
from time import time
from warnings import filterwarnings
filterwarnings("once", category=DeprecationWarning) # Display just the first matching deprecation warnings (multi_class changing to multi_label).

#Load Data

In [ ]:
import pandas as pd
filepath = 'new_data.csv' #missing values have been filled with 'Nothing' while extracting this data from the main patient dataset 
#df = pd.read_csv(filepath)
df = pd.read_csv(filepath, nrows=1000)  #, usecols=['label', 'feedback'])
#df = df.iloc[:500, :]; df.reset_index(inplace=True, drop=True)  #take a small sample to test diffrent nli models for zeroshot

In [ ]:
print(df.shape)
df.head()

(1000, 2)


,label,feedback
0,Couldn't be improved,Nothing.
1,Environment/ facilities,Temperature in theatre a little low.
2,Access,Same service available at Bingham Health Centre.
3,Communication,Appointment details given over phone - no phys...
4,Communication,On one occasion I was not made aware that my a...


# Prepare data for prediction

In [ ]:
sequence = df.feedback.values
candidate_labels = df.label.unique()
sequence
print(len(candidate_labels))
candidate_labels


9


array(["Couldn't be improved", 'Environment/ facilities', 'Access',
       'Communication', 'Dignity', 'Staff', 'Care received',
       'Transition/coordination', 'Miscellaneous'], dtype=object)

#Prediction

In [ ]:
#model="roberta-large-mnli"
models = ['valhalla/distilbart-mnli-12-1', 'valhalla/distilbart-mnli-12-9']
# #models = ['bart-large-mnli', 'roberta-large-mnli', 'typeform/distilbert-base-uncased-mnli', 
#           'facebook/bart-large-mnli', 'vicgalle/xlm-roberta-large-xnli-anli'
#           'valhalla/distilbart-mnli-12-1', 'joeddav/xlm-roberta-large-xnli',
#           'typeform/mobilebert-uncased-mnli', 'valhalla/distilbart-mnli-12-6', 
#           'valhalla/distilbart-mnli-12-9']
#model="distilbert-base-cased"

Any NLI model can be used, but the id of the 'entailment' label must be included in the model config's 
~transformers.PretrainedConfig.label2id.

In [ ]:
%%time
Model_performance_df = pd.DataFrame(columns = ['model_name', 'score', 'model_initialising_time',
                                               'model_prediction_time'])
for j in models:
  model = j
  start = time()
  classifier = pipeline("zero-shot-classification", model=model, device=0) # utilize GPU)
  end = time()
  model_initialising_time = end – start


  #initialise attribute to hold the predicted class and its score
  feedback_predict_score = []; feedback_predict = []

  for i in range(len(df)):
    start = time()
    result = classifier(sequence[i], candidate_labels) # to do multiclass classification set <multi_class=True>
    feedback_predict.append(result['labels'][0])
    feedback_predict_score.append(float('%.2f' %result['scores'][0]))
    end = time()
    model_prediction_time = end – start

  #Model evaluation
  y_true = df.label.values
  y_pred = feedback_predict
  score = accuracy_score(y_true, y_pred)
  Model_performance_df['Model_name'] = model
  Model_performance_df['score'] = float(f'{score:.2f}')
  Model_performance_df['model_initialising_time'] = model_initialising_time
  Model_performance_df['model_prediction_time'] = model_prediction_time

Model_performance_df.to_csv('Models_performance.txt')

CPU times: user 44.5 s, sys: 285 ms, total: 44.8 s
Wall time: 44.4 s


#Model evaluation

In [ ]:
#Model evaluation
from sklearn.metrics import accuracy_score

y_true = df.label.values
y_pred = feedback_predict
score = accuracy_score(y_true, y_pred)
score


0.424

#Track the performance of the various models

In [ ]:
Model_performance = f'{model}: {score:.2f}'
with open('Models_performance.txt', 'a') as file:
  file.write(Model_performance+'\n')

In [ ]:
#investigating the importance of the probability score
df_above60 = df1[df1['feedback_predict_score' ]>0.70]
df_below60 = df1[df1['feedback_predict_score' ]<0.70]

y_true_above60 = df_above60.label.values
y_pred_above60 = df_above60.feedback_predict.values

y_true_below60 = df_below60.label.values
y_pred_below60 = df_below60.feedback_predict.values
print (f'Entries with below 60% probability score: No = {len(df_below60)}, Accuracy = {accuracy_score(y_true_below60, y_pred_below60)}')
print (f'Entries with over 60% probability score: No = {len(df_above60)}, Accuracy = {accuracy_score(y_true_above60, y_pred_above60)}')

# Possible useful codes

In [ ]:
df['feedback_predict'] = feedback_predict
df['feedback_predict_score'] = feedback_predict_score
df1 = df.loc[:, ['feedback','label','feedback_predict', 'feedback_predict_score']]
df1.head()

In [ ]:
# #check for missing values
# print(df.isna().sum())

# #fill missing value has this causes runtime error while fiting the model 
# df['feedback'].fillna('Nothing', inplace = True)
# df.isna().sum()

In [ ]:
#testing 
#trainer.train()
hypothesis_template = "This text is about {}."

b = 212
a = classifier(sequence[b], candidate_labels, hypothesis_template=hypothesis_template) # to do multiclass classification set <multi_class=True>
print (a['labels'], '\n', a['scores'], '\n', sequence[b], '\n', df.label[b])

['Care received', "Couldn't be improved", 'Staff', 'Miscellaneous', 'Access', 'Communication', 'Environment/ facilities', 'Transition/coordination', 'Dignity'] 
 [0.5051387548446655, 0.17052897810935974, 0.08590280264616013, 0.06936632096767426, 0.0408400297164917, 0.04034673422574997, 0.033513203263282776, 0.029891660436987877, 0.02447149157524109] 
 very happy with the service 
 Couldn't be improved
